In [2]:
import boto3
import sagemaker
import datetime as dt
import pandas as pd

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
#Replace this value with the S3 Bucket Created
# default_bucket = "customer-churn-sm-pipeline"
default_bucket = "sagemaker-studio-211125325047-83iipw5p0ug"
s3_uri = "s3://sagemaker-studio-211125325047-83iipw5p0ug/"

In [4]:
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
sklearn_processor_version="0.23-1"
model_package_group_name="ChurnModelPackageGroup"
pipeline_name= "ChurnModelSMPipeline"
clarify_image = sagemaker.image_uris.retrieve(framework='sklearn',version=sklearn_processor_version,region=region)


In [5]:
def preprocess_data(file_path):
    df = pd.read_csv(file_path)
    ## Convert to datetime columns
    df["firstorder"]=pd.to_datetime(df["firstorder"],errors='coerce')
    df["lastorder"] = pd.to_datetime(df["lastorder"],errors='coerce')
    ## Drop Rows with null values
    df = df.dropna()
    ## Create Column which gives the days between the last order and the first order
    df["first_last_days_diff"] = (df['lastorder']-df['firstorder']).dt.days
    ## Create Column which gives the days between when the customer record was created and the first order
    df['created'] = pd.to_datetime(df['created'])
    df['created_first_days_diff']=(df['created']-df['firstorder']).dt.days
    ## Drop Columns
    df.drop(['custid','created','firstorder','lastorder'],axis=1,inplace=True)
    ## Apply one hot encoding on favday and city columns
    df = pd.get_dummies(df,prefix=['favday','city'],columns=['favday','city'])
    return df

In [6]:
baseline_data = preprocess_data(s3_uri + "data/storedata_total.csv")
baseline_data.pop("retained")
baseline_sample = baseline_data.sample(frac=0.0002)

/opt/conda/lib/python3.10/site-packages/fsspec/registry.py:275: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
/tmp/ipykernel_18/4152207287.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["firstorder"]=pd.to_datetime(df["firstorder"],errors='coerce')
/tmp/ipykernel_18/4152207287.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["lastorder"] = pd.to_datetime(df["lastorder"],errors='coerce')
/tmp/ipykernel_18/4152207287.py:11: UserWarning: Could not infer format, so each element 

In [7]:
pd.DataFrame(baseline_sample).to_csv(s3_uri + "data/baseline.csv",header=False,index=False)

In [8]:
batch_data = preprocess_data(s3_uri + "data/storedata_total.csv")
batch_data.pop("retained")
batch_sample = batch_data.sample(frac=0.2)

/tmp/ipykernel_18/4152207287.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["firstorder"]=pd.to_datetime(df["firstorder"],errors='coerce')
/tmp/ipykernel_18/4152207287.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["lastorder"] = pd.to_datetime(df["lastorder"],errors='coerce')
/tmp/ipykernel_18/4152207287.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['created'] = pd.to_datetime(df['created'])


In [9]:
pd.DataFrame(batch_sample).to_csv(s3_uri + "data/batch.csv",header=False,index=False)

In [10]:
from pipelines.customerchurn.pipeline import get_pipeline

pipeline = get_pipeline(
    region = region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
    custom_image_uri=clarify_image,
    sklearn_processor_version=sklearn_processor_version
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.0.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [11]:
pipeline.definition()

'{"Version": "2020-12-01", "Metadata": {}, "Parameters": [{"Name": "ProcessingInstanceType", "Type": "String", "DefaultValue": "ml.m5.xlarge"}, {"Name": "ProcessingInstanceCount", "Type": "Integer", "DefaultValue": 1}, {"Name": "TrainingInstanceType", "Type": "String", "DefaultValue": "ml.m5.xlarge"}, {"Name": "InputData", "Type": "String", "DefaultValue": "s3://sagemaker-studio-211125325047-83iipw5p0ug/data/storedata_total.csv"}, {"Name": "BatchData", "Type": "String", "DefaultValue": "s3://sagemaker-studio-211125325047-83iipw5p0ug/data/batch/batch.csv"}], "PipelineExperimentConfig": {"ExperimentName": {"Get": "Execution.PipelineName"}, "TrialName": {"Get": "Execution.PipelineExecutionId"}}, "Steps": [{"Name": "ChurnModelProcess", "Type": "Processing", "Arguments": {"ProcessingResources": {"ClusterConfig": {"InstanceType": {"Get": "Parameters.ProcessingInstanceType"}, "InstanceCount": {"Get": "Parameters.ProcessingInstanceCount"}, "VolumeSizeInGB": 30}}, "AppSpecification": {"ImageUri

In [12]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-2:211125325047:pipeline/ChurnModelSMPipeline',
 'ResponseMetadata': {'RequestId': '670bfdde-ea0e-48b5-b512-4b47513cf8d0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '670bfdde-ea0e-48b5-b512-4b47513cf8d0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '88',
   'date': 'Mon, 08 Jul 2024 01:29:03 GMT'},
  'RetryAttempts': 0}}

In [16]:
execution = pipeline.start()

In [17]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-2:211125325047:pipeline/ChurnModelSMPipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-2:211125325047:pipeline/ChurnModelSMPipeline/execution/66zt5bl5oh6r',
 'PipelineExecutionDisplayName': 'execution-1720402427443',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'churnmodelsmpipeline',
  'TrialName': '66zt5bl5oh6r'},
 'CreationTime': datetime.datetime(2024, 7, 8, 1, 33, 47, 317000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 7, 8, 1, 33, 47, 317000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-2:211125325047:user-profile/d-tswep4shqtam/default-20240520t104932',
  'UserProfileName': 'default-20240520t104932',
  'DomainId': 'd-tswep4shqtam',
  'IamIdentity': {'Arn': 'arn:aws:sts::211125325047:assumed-role/AmazonSageMaker-ExecutionRole-20240520T104933/SageMaker',
   'PrincipalId': 'AROATCKAMZT33MHHLHIVU:SageMaker'}},
 'LastModifiedBy': {'U

In [19]:
execution.list_steps()

[{'StepName': 'ChurnModelProcess',
  'StartTime': datetime.datetime(2024, 7, 8, 1, 33, 49, 5000, tzinfo=tzlocal()),
  'StepStatus': 'Executing',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-2:211125325047:processing-job/pipelines-66zt5bl5oh6r-ChurnModelProcess-sBolciFGso'}},
  'AttemptCount': 1}]